In [1]:
from bs4 import BeautifulSoup
import requests
import re
import datetime
from tqdm import tqdm
import sys
from typing import List

def makePgNum(num):
    if num == 1:
        return num
    elif num == 0:
        return num+1
    else:
        return num+9*(num-1)

def makeUrl(search, start_pg, end_pg):
    if start_pg == end_pg:
        start_page = makePgNum(start_pg)
        url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + "&start=" + str(start_page)
        print("생성url: ", url)
        return url
    else:
        urls = []
        for i in range(start_pg, end_pg + 1):
            page = makePgNum(i)
            url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + "&start=" + str(page)
            urls.append(url)
        print("생성url: ", urls)
        return urls    

def news_attrs_crawler(articles,attrs):
    attrs_content=[]
    for i in articles:
        attrs_content.append(i.attrs[attrs])
    return attrs_content

headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}

def articles_crawler(url:str) -> List[str]:
    #html 불러오기
    original_html = requests.get(url,headers=headers)
    html = BeautifulSoup(original_html.text, "html.parser")
    # 모든 링크 수집
    all_links = [a.get("href") for a in html.find_all("a", href=True)]
    # 뉴스 링크 필터링
    news_links = [link for link in all_links if "n.news.naver.com" in link]
    news_links = list(set(news_links))  # 중복 제거

    return news_links



In [2]:
search = input("검색할 키워드를 입력해주세요:")
page = int(input("\n크롤링할 시작 페이지를 입력해주세요. ex)1(숫자만입력):")) # ex)1 =1페이지,2=2페이지...
print("\n크롤링할 시작 페이지: ",page,"페이지")

page2 = int(input("\n크롤링할 종료 페이지를 입력해주세요. ex)1(숫자만입력):")) # ex)1 =1페이지,2=2페이지...
print("\n크롤링할 종료 페이지: ",page2,"페이지")


크롤링할 시작 페이지:  1 페이지

크롤링할 종료 페이지:  2 페이지


In [3]:
# URL 생성
url_list = makeUrl(search, page, page2)

# 결과 저장 리스트 초기화
news_titles = []
news_url_nested = []  # 중첩 리스트
news_contents = []

# 뉴스 URL 수집
for single_url in url_list:
    result = articles_crawler(single_url)
    news_url_nested.append(result)

# 중첩 리스트 평탄화 함수 정의
def makeList(newlist, content):
    for i in content:
        for j in i:
            newlist.append(j)
    return newlist

# 평탄화 실행
news_url_flat = []
makeList(news_url_flat, news_url_nested)





생성url:  ['https://search.naver.com/search.naver?where=news&sm=tab_pge&query=물류&start=1', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=물류&start=11']


['https://n.news.naver.com/mnews/article/092/0002373588?sid=105',
 'https://n.news.naver.com/mnews/article/003/0013230968?sid=102',
 'https://n.news.naver.com/mnews/article/421/0008242290?sid=102',
 'https://n.news.naver.com/mnews/article/008/0005191743?sid=101',
 'https://n.news.naver.com/mnews/article/366/0001075629?sid=101',
 'https://n.news.naver.com/mnews/article/015/0005129772?sid=101',
 'https://n.news.naver.com/mnews/article/277/0005589752?sid=101',
 'https://n.news.naver.com/mnews/article/421/0008239484?sid=101',
 'https://n.news.naver.com/mnews/article/003/0013233630?sid=102',
 'https://n.news.naver.com/mnews/article/003/0013230611?sid=101',
 'https://n.news.naver.com/mnews/article/018/0006010365?sid=101',
 'https://n.news.naver.com/mnews/article/030/0003310729?sid=101',
 'https://n.news.naver.com/mnews/article/448/0000526784?sid=102',
 'https://n.news.naver.com/mnews/article/214/0001423415?sid=102',
 'https://n.news.naver.com/mnews/article/001/0015380072?sid=102',
 'https://

In [5]:
# 네이버 뉴스 URL만 필터링
final_urls = []
for i in tqdm(range(len(news_url_flat))):
    if "n.news.naver.com" in news_url_flat[i]:
        final_urls.append(news_url_flat[i])
    else:
        pass


100%|██████████| 42/42 [00:00<00:00, 289738.11it/s]


In [ ]:

# 검색할 키워드를 입력해주세요:물류
# 크롤링할 시작 페이지를 입력해주세요.
# 크롤링할 시작 페이지: 1 페이지
# 크롤링할 종료 페이지를 입력해주세요.
# 크롤링할 종료 페이지: 3 페이지

In [6]:
for i in tqdm(final_urls):
    
    news = requests.get(i, headers=headers)
    news_html = BeautifulSoup(news.text, "html.parser")

    for tag in news_html.select("em.img_desc"):
        tag.decompose()

    # 제목 크롤링
    title = news_html.select_one("#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
    if title is None:
        title = news_html.select_one("#content > div.end_ct > div > h2")
    title = title.get_text(strip=True) if title else "제목 없음"

    # 본문 크롤링
    content = news_html.select_one("article#dic_area") or news_html.select_one("#articeBody")
    if content:
        content = content.get_text(separator="\n", strip=True)
    else:
        content = "본문 없음"

    # 태그 제거 및 전처리
    pattern1 = '<[^>]*>'
    title = re.sub(pattern=pattern1, repl='', string=str(title))
    content = re.sub(pattern=pattern1, repl='', string=content)

    content = re.sub(r"\n{3,}", "\n\n", content).strip()
    content = re.sub(r"<!--.*?-->", "", content, flags=re.DOTALL)


    # 리스트에 저장
    news_titles.append(title)
    news_contents.append(content)


100%|██████████| 42/42 [00:11<00:00,  3.76it/s]


In [7]:
print("검색된 기사 갯수: 총 ",len(final_urls),'개')
print("\n[뉴스 제목]")
print(news_titles)
print("\n[뉴스 링크]")
print(final_urls)
print("\n[뉴스 내용]")
print(news_contents)
print('news_title: ',len(news_titles))
print('news_url: ',len(final_urls))
print('news_contents: ',len(news_contents))

검색된 기사 갯수: 총  42 개

[뉴스 제목]
['DN솔루션즈, 천안에 700평 물류센터 신설', '창원산단 DN솔루션즈, 천안에 부품 물류센터 개소', '물류센터 출퇴근용 버스, 도로 경계석 들이받아 7명 부상', "빙그레, 신임 대표에 물류회사 '제때' 김광수 대표 내정", 'DN솔루션즈, 천안 물류센터 신설', 'DN솔루션즈, 천안에 대규모 부품 물류센터 신설', '배민 "미래 물류혁신을 이끌 인재 모십니다"', "서울역 '짐보관·배송' 한 달 만에 8000건…철도 물류 서비스 진화", '제주, 물동량 산정 용역 착수…"영세 물류산업 지원"', '"물류혁신 이끌 인재 모집" 우아한청년들, 국군장병 취업박람회 참가', 'DN솔루션즈, 천안 물류센터 신설', '우아한청년들, 국군장병 취업박람회 참가…“물류혁신 이끌 인재 모집”', '인천서 물류센터 퇴근 버스, 경계석 충돌…7명 부상', '인천 서구서 물류센터 출근 버스가 도로 연석 들이받아‥7명 부상', '인천서 물류센터 출퇴근용 버스 도로 경계석 충돌…7명 부상', 'DN솔루션즈, 천안 물류센터 개소…서비스 엔지니어 채용도', "'SKT 유심 해킹사태' 유통·물류업계 보안강화", 'CJ대한통운, 사우디에 배송 네트워크 구축…"중동 물류 확장"', '인하대 물류전문대학원, 가을학기 신입생 모집', '인하대 물류전문대학원 2025학년도 가을학기 신입생 모집', '‘국내 최초·유일’ 인하대 물류대학원, 가을학기 신입생 모집', "인하대 물류전문대학원, '2025  가을학기 신입생' 모집", 'HD현대, 글로벌 해운사 머스크와 탈탄소 기술·통합물류 협력', 'HD현대, 머스크와 ‘탈탄소 기술·물류’ 협력 합의', '용인시산업진흥원, 수출 중소기업 대상 해외물류비 3차 지원사업 추진', "SKT 사태에 유통·물류업계도 '보안강화'", 'CJ대한통운, 중동지역 배송 네트워크 구축…"물류영토 확장"', "평창군, 농공단지 입주기업 '물류비 최대 2천만원' 지원", "서울역 '짐보관·배송' 한 달 만에 8000건…철